In [19]:
import pandas as pd
import numpy as np

train = pd.read_csv("../data/train.csv")

In [20]:
interactions = (
    train.groupby(["msno", "song_id"])
         .size()
         .reset_index(name="play_count")
)

In [21]:
user_codes = interactions["msno"].astype("category").cat.codes
item_codes = interactions["song_id"].astype("category").cat.codes

user_id_map = dict(enumerate(interactions["msno"].astype("category").cat.categories))
item_id_map = dict(enumerate(interactions["song_id"].astype("category").cat.categories))


In [22]:
user_mapping = dict(enumerate(interactions["msno"].astype("category").cat.categories))
item_mapping = dict(enumerate(interactions["song_id"].astype("category").cat.categories))

In [23]:
from scipy.sparse import csr_matrix

user_item_matrix = csr_matrix(
    (interactions["play_count"],
     (user_codes, item_codes))
)

In [24]:
user_item_matrix.shape

(30755, 359966)

In [ ]:
from implicit.als import AlternatingLeastSquares

als_model = AlternatingLeastSquares(
    factors=64,
    regularization=0.1,
    iterations=20,
    random_state=42
)

als_model.fit(user_item_matrix)


100%|██████████| 20/20 [00:46<00:00,  2.35s/it]


: 

In [ ]:
als_model.fit(user_item_matrix.T)

/Users/aditisonkar12/Desktop/Python Projects/Music-recommendation-system/venv/lib/python3.10/site-packages/implicit/utils.py:164: ParameterWarning: Method expects CSR input, and was passed csc_matrix instead. Converting to CSR took 0.03464984893798828 seconds
  warnings.warn(
  0%|          | 0/20 [00:00<?, ?it/s]

In [ ]:
user_id = 0
user_items = user_item_matrix[user_id]

ids, scores = als_model.recommend(
    userid=user_id,
    user_items=user_items,
    N=10
)

ids, scores


(array([9, 8, 7, 6, 5, 4, 3, 2, 1, 0], dtype=int32),
 array([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan], dtype=float32))

In [ ]:
print(user_item_matrix.shape)
print(als_model.item_factors.shape)


(30755, 359966)
(359966, 64)


In [ ]:
import os
import joblib
from scipy.sparse import save_npz

In [ ]:
MODEL_DIR = "../model"
os.makedirs(MODEL_DIR, exist_ok=True)

In [ ]:
joblib.dump(als_model, f"{MODEL_DIR}/als_model.pkl")

['../model/als_model.pkl']

In [ ]:
save_npz(f"{MODEL_DIR}/user_item_matrix.npz", user_item_matrix)

In [ ]:
joblib.dump(user_id_mapping, f"{MODEL_DIR}/user_id_mapping.pkl")
joblib.dump(item_id_mapping, f"{MODEL_DIR}/item_id_mapping.pkl")
joblib.dump(id_to_song, f"{MODEL_DIR}/id_to_song.pkl")

NameError: name 'user_id_mapping' is not defined

In [ ]:
# user mapping: original msno -> internal id
user_id_mapping = dict(zip(interactions["msno"].unique(),
                           range(interactions["msno"].nunique())))

# item mapping: original song_id -> internal id
item_id_mapping = dict(zip(interactions["song_id"].unique(),
                           range(interactions["song_id"].nunique())))

# reverse mapping: internal item id -> song_id
id_to_song = {v: k for k, v in item_id_mapping.items()}